In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow import keras  
from tensorflow.keras import layers 

In [ ]:
df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df.drop('id',axis='columns',inplace=True)
y_fourier = np.load('../input/coeficientes-0/Coef_2.npy')

In [ ]:
df.head()

In [ ]:
df2 = df[df['u_out']==0]
df2.reset_index(drop=True,inplace=True)
len(df2)

In [ ]:
s = df2.groupby('breath_id')
lens = []
for breath,table in s:
    if len(table)<29:
        lens.append(breath)
for i in range(len(lens)):
    df2 = df2[df2.breath_id != lens[i]]
df2.reset_index(drop=True,inplace=True)
df2.breath_id.value_counts().value_counts()

In [ ]:
s = df2.groupby('breath_id')
lens = []
for breath,table in s:
    if len(table) > 29:
        for i in table.tail(len(table)-29).index:
            lens.append(i)

In [ ]:
df2.drop(lens,inplace = True)
df2.reset_index(drop=True,inplace=True)
df2.breath_id.value_counts().value_counts()

In [ ]:
df2.breath_id.value_counts().value_counts()

In [ ]:
df = df2.copy()

In [ ]:
df.R.value_counts()

In [ ]:
x = df.drop(['pressure','breath_id'],axis='columns').to_numpy().reshape(len(df.breath_id.unique()),29,5)
y = df.pressure.to_numpy().reshape(len(df.breath_id.unique()),29)
x.shape
x.shape

**Definimos una funcion para encontrar la serie de fourier compleja**

In [ ]:
"""
import math
from math import pi
def integrar(t,L,ht,n,y):
    integral=0
    while t<L:
        integral=integral+complex(t,y[int(t/ht)])*complex(math.cos(-2*pi*n*t/L),math.sin(-2*pi*n*t/L))*ht/L                                                  
        t+=ht
    return integral

def Fourier_serie(n_terms,time_serie,y):
    i=0
    t=0 #variable de tiempo
    L=time_serie #periodo
    ht=L/80
    lista=[]
    n=int((n_terms)/2)
    i=-n
    ind=0
    while i<=n:
        lista.append(integrar(t,L,ht,i,y))
        t=0
        i+=1
    return(np.array(lista))
"""

In [ ]:
import math
from math import pi
def grafica(Cn,time,pressure):
    L=max(time)
    pressure_serie = []
    for t in time:
        index = len(Cn)-1
        suma = 0
        for n in np.arange(-int(len(Cn)/2),int(len(Cn)/2)+1):
            suma = suma + Cn[index]*complex(math.cos(2*pi*n*t/L),math.sin(-2*pi*n*t/L))
            index = index -1
        pressure_serie.append(suma.imag)
    plt.rcParams["figure.figsize"] = (10, 4)
    plt.plot(time,pressure,color='red')
    plt.plot(time,pressure_serie,color='blue')
    plt.xlim((0,2.6))

In [ ]:
i = 20000
grafica(y_fourier[i],x[i,:,2],y[i,:])

In [ ]:
#x_1 = df.drop(['pressure','breath_id','time_step'],axis='columns').to_numpy().reshape(len(df.breath_id.unique()),80,5)
x_1 = df.drop(['pressure','breath_id','time_step'],axis='columns')
x_1.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_1[x_1.columns] = scaler.fit_transform(x_1[x_1.columns])
x_1.head()

In [ ]:
x_scaled = x_1.to_numpy().reshape(len(df.breath_id.unique()),29,4)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y_fourier,train_size=0.9)
x_train.shape,y_train.shape

In [ ]:
y_train_scaled,y_test_scaled = y_train/np.amax(y_fourier),y_test/np.amax(y_fourier)
#y_train_scaled,y_test_scaled = y_train,y_test

In [ ]:
"""
mod_test = np.absolute(y_test)
ang_test = np.angle(y_test)
mod_train = np.absolute(y_train)
ang_train = np.angle(y_train)
for i in range(len(ang_test)):
    for j in range(len(ang_test[i])):
        if ang_test[i][j] < 0:
            ang_test[i][j] = ang_test[i][j] + 2*np.pi
for i in range(len(ang_train)):
    for j in range(len(ang_train[i])):
        if ang_train[i][j] < 0:
            ang_train[i][j] = ang_train[i][j] + 2*np.pi
mod_max = max(np.amax(mod_test),np.amax(mod_train))
mod_test,mod_train = mod_test/mod_max,mod_train/mod_max
ang_max = max(np.amax(ang_test),np.amax(ang_train))
ang_test,ang_train = ang_test/ang_max,ang_train/ang_max
"""

In [ ]:
x_train_flaten = np.concatenate((x_train[:,:,2:].reshape(len(x_train),2*29),x_train[:,2,:2]),axis=1)
x_test_flaten = np.concatenate((x_test[:,:,2:].reshape(len(x_test),2*29),x_test[:,2,:2]),axis=1)
y_test_flaten = np.concatenate((y_test_scaled.real,y_test_scaled.imag),axis=1)
y_train_flaten = np.concatenate((y_train_scaled.real,y_train_scaled.imag),axis=1)

In [ ]:
"""
model = keras.Sequential([
    layers.Dense(100,input_shape = (2*29+2,),activation='relu'),
    layers.Dense(150,activation='linear' ),
    layers.Dense(200,activation='linear' ),
    layers.Dense(120,activation='linear' ),
    layers.Dense(62,activation='linear')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredLogarithmicError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)
model.fit(x_train_flaten,y_train_flaten,epochs=100)
"""
model = keras.Sequential([
    layers.Dense(500,input_shape = (2*29+2,),activation='relu'),
    layers.Dense(750,activation= 'relu' ),
    layers.Dense(500,activation='linear' ),
    layers.Dense(250,activation='linear' ),
    layers.Dense(62,activation='linear')
])

"""
model = Sequential([
    Input(shape=(60251,60),),
    Bidirectional(LSTM(200, return_sequences=True)),
    Bidirectional(LSTM(150, return_sequences=True)),
    Bidirectional(LSTM(100, return_sequences=True)),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(1),
    ])
"""
opt_sgd = tf.keras.optimizers.SGD(
    learning_rate=0.02, momentum=0.1, nesterov=False, name="SGD")
opt_rms = tf.keras.optimizers.RMSprop(
    learning_rate=0.0015, momentum=0)

model.compile(
    optimizer= "adam",
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)
model.fit(x_train_flaten,y_train_flaten,epochs=100)

In [ ]:
model.evaluate(x_test_flaten,y_test_flaten)

In [ ]:
y_predicted = model.predict(x_test_flaten)
y_predicted.shape,y_test_flaten.shape

In [ ]:
i=13
print(y_predicted[0,i],y_test_flaten[0,i])

### **en la red neuronal meti los coeficientes en un arreglo de una dimension---los pimeros 101 elementos es la parte real de los coeficientes y los otros 101 son la parte imaginaria de los coeficientes**
para poder graficar necesito poner los coeficientes predecidos por el modelo en su forma compleja eso es en lo que ando

In [ ]:
y_predicted.shape

In [ ]:
y_final_predicted = y_predicted[:,:31] + 1j*y_predicted[:,31:]

In [ ]:
y_final_predicted[0].shape


In [ ]:
def grafica_2(Cn,time,Cn_2):
    L=max(time)
    pressure_serie_1 = []
    pressure_serie_2 = []
    for t in time:
        index = len(Cn)-1
        suma = 0
        suma_2 = 0
        for n in np.arange(-int(len(Cn)/2),int(len(Cn)/2)+1):
            suma = suma + Cn[index]*complex(math.cos(2*pi*n*t/L),math.sin(-2*pi*n*t/L))
            suma_2 = suma_2 + Cn_2[index]*complex(math.cos(2*pi*n*t/L),math.sin(-2*pi*n*t/L))
            index = index -1
        pressure_serie_1.append(suma.imag)
        pressure_serie_2.append(suma_2.imag)
    plt.rcParams["figure.figsize"] = (10, 4)
    plt.plot(time,pressure_serie_1,color='red',label = 'Original')
    plt.plot(time,pressure_serie_2,color='blue',label = 'predicted')
    plt.xlim((0,L))
    plt.legend()

In [ ]:
i = 3000

grafica_2(y_final_predicted[i]*np.amax(y_fourier),np.linspace(0,1,80),y_test_scaled[i]*np.amax(y_fourier))

In [ ]:
y_test[i].shape